# Point-Spread Deconvolution of SDO/AIA images
## Mark Cheung, cheung@lmsal.com
### Lessons learned here
1. How to query JSOC for an SDO/AIA level 1 EUV image.
2. How to load the header and the image from the compressed FITS file, and cutout a portion of the image.
3. How to remove the point-spread function on a level 1 image cutout, and compare before / after.

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from aia_sparse_deconvolve import *
from sklearn import linear_model
import drms
import os

### Query the SDO Joint Science Operations Center (JSOC) for some AIA images

In [2]:
jsoc = drms.Client()
ds = jsoc.series(r"aia.lev1_euv_12s")
si = jsoc.info(ds[0])
print(si.segments)
wavelnth = 131
k, s = jsoc.query('{0:s}[2011-02-15T02:30/1m][WAVELNTH = {1:03d}]'.format(ds[0],wavelnth),key='DATE',seg='image')
files = ['http://jsoc.stanford.edu'+image for image in s.image]
print(files)

#Alternatively search for files on the local filesystem
#import glob
#files = glob.glob("/Users/cheung/AIA/ValentinesDayFlare_ribbons/*.*131*.fits")
#files = glob.glob("/Users/cheung/AIA/GST/*.*171*.fits")

       type units       protocol  dims                    note
name                                                          
image   int  None  link via lev1  None       AIA level 1 image
spikes  int  None  link via lev1  None  Cosmic ray information
['http://jsoc.stanford.edu/SUM78/D137194136/S00000/image_lev1.fits', 'http://jsoc.stanford.edu/SUM74/D137194134/S00000/image_lev1.fits', 'http://jsoc.stanford.edu/SUM80/D137194351/S00000/image_lev1.fits', 'http://jsoc.stanford.edu/SUM74/D137194392/S00000/image_lev1.fits', 'http://jsoc.stanford.edu/SUM79/D137194429/S00000/image_lev1.fits']


//anaconda/envs/sdo2018/lib/python3.6/site-packages/drms/client.py:1089: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res_key = pd.DataFrame.from_items(zip(names, values))
//anaconda/envs/sdo2018/lib/python3.6/site-packages/drms/client.py:1100: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res_seg = pd.DataFrame.from_items(zip(names, values))


### Set size of image cutout, and load corresponding Point Spread Function (PSF).
### For the time being, PSF files for 192x192 cutouts available.

In [6]:
# Image cutout size and center 
nx = int(192) 
ny = int(192) 
bit_depth = 14.0

prefix = "psf_sparse_entrance_filter_only" # This PSF doesn't include CCD charge-spreading
#prefix = "psf_sparse" # This PSF includes CCD charge-spreading
pfile = "./response/PSF/{0}_{1:03d}_{2:03d}x{3:03d}.npz".format(prefix,wavelnth, nx, ny)
restore = True
rest = np.load(pfile)
C = rest['C'].tolist()
# The PSF is stored in a sparse format, which will be passed on the the deconvolution routine.
d_array = rest['d_array']
i_array = rest['i_array']
j_array = rest['j_array']

# Cinv is the straight matrix inverse
# The reason we convert C to dense form first is because it seems to be faster for computing Cinv
# But for lasso it seems faster to have C in sparse form. 
# Cinv = np.linalg.inv(np.array(C.todense()))

### Set center of cutout region, and loop through list of files. 

In [ ]:
ycenter=1660
xcenter=2380
#Format of output image names
outfile = 'deconvolved_{0:03d}_{1:03d}.png'

count=0
for f in files:
    dummy, b = fits.open(f)
    b.verify('fix')
    data = np.array(b.data.T, dtype=float)
    h = b.header
    exptime = h['EXPTIME']

    obj = data[int(xcenter-nx/2):int(xcenter+nx/2),int(ycenter-ny/2):int(ycenter+ny/2)]
    img = np.array(obj)
    sat = img
    # The next line does the PSF deconvolution
    desat, model = sparse_deconv(sat, d_array, i_array, j_array, bit_depth,gap=0.3, alpha=1e-7)

    # The following lines plot the before and after images.
    images = (img[:,:]/exptime,  desat[:,:]/exptime)
    colorbar_label='Log2 [DN/s]'
    labels = ('AIA {}@{}'.format(wavelnth,h['DATE-OBS'][0:19]),'desat, exptime={0:.2f}'.format(exptime))
    types = ('plot','plot')
    if wavelnth == 193:
        clim = (2,20)
    else:
        clim = (2,18)
    compare_imgs(images,labels,types,clim=clim, cmap='nipy_spectral',
                 savefig=outfile.format(wavelnth,count),
                 colorbar_label=colorbar_label)
    count+=1


In [8]:
#nx = 192
#ny = 192
#for wavelnth in (94,131,171,193,211,335,304):
#    pfile = "/Users/cheung/AIA/weber_psfs_rc/psf_sparse_{0:03d}_{1:03d}x{2:03d}.npz".format(wavelnth, nx, ny)
#    C, d_array, i_array, j_array = read_psf(wavelnth,(nx,ny),dir='/Users/cheung/AIA/weber_psfs_rc') 
#    np.savez_compressed(pfile,C=C, d_array=d_array, i_array=i_array, j_array=j_array, nx=nx, ny=ny)

In [9]:
#nx = 192
#ny = 192
#for wavelnth in (94,131,171,193,211,335,304):
#    pfile = "/Users/cheung/AIA/weber_psfs_rc/psf_sparse_entrance_filter_only_{0:03d}_{1:03d}x{2:03d}.npz".format(wavelnth, nx, ny)
#    C, d_array, i_array, j_array = read_psf(wavelnth,(nx,ny),dir='/Users/cheung/AIA/weber_psfs_rc', nocore=True) 
#    np.savez_compressed(pfile,C=C, d_array=d_array, i_array=i_array, j_array=j_array, nx=nx, ny=ny)